In [9]:
import traceback
import cv2
import numpy as np
import math


cap = cv2.VideoCapture("result.avi")
     
while(1):
        
    try:  #an error comes if it does not find anything in window as it cannot find contour of max area
          #therefore this try error statement
          
        ret, frame = cap.read()
        frame = cv2.flip(frame,1)
        kernel = np.ones((3,3),np.uint8)
        
        #define region of interest
        roi = frame[0:150, 0:150]
        
        
        cv2.rectangle(frame,(0,0),(150,150),(0,255,0),0)    
        hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
        
        
         
    # define range of skin color in HSV
        lower_skin = np.array([0,20,70], dtype=np.uint8)
        upper_skin = np.array([20,255,255], dtype=np.uint8)
        
     #extract skin colur imagw  
        mask = cv2.inRange(hsv, lower_skin, upper_skin)
        
   
        
    #extrapolate the hand to fill dark spots within
        mask = cv2.dilate(mask,kernel,iterations = 4)
        
    #blur the image
        mask = cv2.GaussianBlur(mask,(5,5),100) 
     
    #find contours
        contours,hierarchy= cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        print(contours)
        print(hierarchy)
   #find contour of max area(hand)
        cnt = max(contours, key = lambda x: cv2.contourArea(x))
        
    #approx the contour a little
        epsilon = 0.0005*cv2.arcLength(cnt,True)
        approx= cv2.approxPolyDP(cnt,epsilon,True)
       
        
    #make convex hull around hand
        hull = cv2.convexHull(cnt)
        
     #define area of hull and area of hand
        areahull = cv2.contourArea(hull)
        areacnt = cv2.contourArea(cnt)
      
    #find the percentage of area not covered by hand in convex hull
        arearatio=((areahull-areacnt)/areacnt)*100
    
     #find the defects in convex hull with respect to hand
        hull = cv2.convexHull(approx, returnPoints=False)
        defects = cv2.convexityDefects(approx, hull)
        
    # l = no. of defects
        l=0
        
    #code for finding no. of defects due to fingers
        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
            start = tuple(approx[s][0])
            end = tuple(approx[e][0])
            far = tuple(approx[f][0])
            pt= (100,180)
            
            
            # find length of all sides of triangle
            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
            c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
            s = (a+b+c)/2
            ar = math.sqrt(s*(s-a)*(s-b)*(s-c))
            
            #distance between point and convex hull
            d=(2*ar)/a
            
            # apply cosine rule here
            angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
            
        
            # ignore angles > 90 and ignore points very close to convex hull(they generally come due to noise)
            if angle <= 90 and d>30:
                l += 1
                cv2.circle(roi, far, 3, [255,0,0], -1)
            
            #draw lines around hand
            cv2.line(roi,start, end, [0,255,0], 2)
            
            
        l+=1
        
        #print corresponding gestures which are in their ranges
        font = cv2.FONT_HERSHEY_SIMPLEX
        if l==1:
            if areacnt<2000:
                cv2.putText(frame,'Put hand in the box',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            else:
                if arearatio<12:
                    cv2.putText(frame,'0',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                elif arearatio<17.5:
                    cv2.putText(frame,'Best of luck',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                   
                else:
                    cv2.putText(frame,'1',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                    
        elif l==2:
            cv2.putText(frame,'2',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            
        elif l==3:
         
              if arearatio<27:
                    cv2.putText(frame,'3',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
              else:
                    cv2.putText(frame,'ok',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                    
        elif l==4:
            cv2.putText(frame,'4',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            
        elif l==5:
            cv2.putText(frame,'5',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            
        elif l==6:
            cv2.putText(frame,'reposition',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            
        else :
            cv2.putText(frame,'reposition',(10,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            
        #show the windows
        cv2.imshow('mask',mask)
        cv2.imshow('frame',frame)
    except Exception:
        traceback.print_exc()
        pass
       # break
        
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
    
cv2.destroyAllWindows()
cap.release() 

[array([[[ 0, 42]],

       [[ 0, 85]],

       [[ 7, 85]],

       [[ 7, 83]],

       [[ 8, 82]],

       [[10, 82]],

       [[10, 81]],

       [[11, 80]],

       [[11, 66]],

       [[12, 65]],

       [[12, 62]],

       [[13, 61]],

       [[13, 42]]], dtype=int32), array([[[130,  27]],

       [[130,  33]],

       [[129,  34]],

       [[125,  34]],

       [[125,  35]],

       [[124,  36]],

       [[124,  44]],

       [[123,  45]],

       [[122,  45]],

       [[122,  49]],

       [[120,  51]],

       [[120,  55]],

       [[119,  56]],

       [[117,  56]],

       [[115,  58]],

       [[112,  58]],

       [[111,  59]],

       [[110,  59]],

       [[107,  62]],

       [[107,  63]],

       [[102,  68]],

       [[102,  69]],

       [[ 99,  72]],

       [[ 98,  72]],

       [[ 97,  73]],

       [[ 96,  73]],

       [[ 95,  74]],

       [[ 93,  74]],

       [[ 92,  75]],

       [[ 92,  93]],

       [[ 90,  95]],

       [[ 90,  96]],

       [[ 89,  97]],


[array([[[126,  90]],

       [[126,  91]],

       [[125,  92]],

       [[122,  92]],

       [[122,  93]],

       [[121,  94]],

       [[120,  94]],

       [[120,  95]],

       [[119,  96]],

       [[114,  96]],

       [[114,  97]],

       [[113,  98]],

       [[112,  98]],

       [[109, 101]],

       [[106, 101]],

       [[106, 114]],

       [[105, 115]],

       [[105, 117]],

       [[104, 118]],

       [[104, 133]],

       [[106, 133]],

       [[107, 134]],

       [[109, 134]],

       [[110, 135]],

       [[112, 135]],

       [[113, 136]],

       [[116, 136]],

       [[117, 137]],

       [[119, 137]],

       [[120, 138]],

       [[120, 145]],

       [[121, 145]],

       [[122, 146]],

       [[122, 149]],

       [[143, 149]],

       [[143, 148]],

       [[144, 147]],

       [[149, 147]],

       [[149, 110]],

       [[148, 110]],

       [[147, 109]],

       [[147, 106]],

       [[144, 106]],

       [[143, 105]],

       [[143,  98]],

       [[

[[[ 1 -1 -1 -1]
  [ 2  0 -1 -1]
  [ 3  1 -1 -1]
  [ 4  2 -1 -1]
  [ 5  3 -1 -1]
  [ 6  4 -1 -1]
  [-1  5 -1 -1]]]
[array([[[120, 138]],

       [[120, 149]],

       [[149, 149]],

       [[149, 138]]], dtype=int32), array([[[ 16, 132]],

       [[ 16, 137]],

       [[ 15, 138]],

       [[  3, 138]],

       [[  1, 140]],

       [[  0, 140]],

       [[  0, 149]],

       [[ 59, 149]],

       [[ 59, 136]],

       [[ 52, 136]],

       [[ 50, 134]],

       [[ 36, 134]],

       [[ 35, 133]],

       [[ 35, 132]]], dtype=int32), array([[[142,  68]],

       [[142,  81]],

       [[149,  81]],

       [[149,  68]]], dtype=int32), array([[[ 88,  42]],

       [[ 88,  54]],

       [[ 91,  54]],

       [[ 93,  56]],

       [[ 93,  57]],

       [[101,  57]],

       [[102,  58]],

       [[102,  59]],

       [[120,  59]],

       [[120,  58]],

       [[121,  57]],

       [[123,  57]],

       [[123,  42]]], dtype=int32), array([[[28, 26]],

       [[28, 31]],

       [[27, 32]],


[array([[[ 28, 134]],

       [[ 28, 149]],

       [[ 40, 149]],

       [[ 40, 134]]], dtype=int32), array([[[ 34, 106]],

       [[ 34, 123]],

       [[ 49, 123]],

       [[ 49, 110]],

       [[ 48, 110]],

       [[ 47, 109]],

       [[ 47, 106]]], dtype=int32), array([[[ 98,   8]],

       [[ 98,   9]],

       [[ 97,  10]],

       [[ 90,  10]],

       [[ 90,  13]],

       [[ 89,  14]],

       [[ 89,  15]],

       [[ 88,  16]],

       [[ 88,  37]],

       [[ 89,  37]],

       [[ 90,  38]],

       [[ 90,  41]],

       [[ 89,  42]],

       [[ 86,  42]],

       [[ 86,  43]],

       [[ 85,  44]],

       [[ 85,  45]],

       [[ 84,  46]],

       [[ 84,  57]],

       [[ 83,  58]],

       [[ 80,  58]],

       [[ 80,  71]],

       [[ 79,  72]],

       [[ 76,  72]],

       [[ 76,  73]],

       [[ 75,  74]],

       [[ 75,  81]],

       [[ 74,  82]],

       [[ 64,  82]],

       [[ 64,  95]],

       [[ 76,  95]],

       [[ 77,  94]],

       [[ 77,  91]],

   

[array([[[ 96, 106]],

       [[ 96, 119]],

       [[ 97, 120]],

       [[ 97, 130]],

       [[101, 130]],

       [[102, 131]],

       [[102, 138]],

       [[113, 138]],

       [[114, 139]],

       [[117, 139]],

       [[118, 140]],

       [[119, 140]],

       [[122, 143]],

       [[137, 143]],

       [[137, 141]],

       [[138, 140]],

       [[139, 140]],

       [[139, 122]],

       [[136, 122]],

       [[135, 121]],

       [[135, 118]],

       [[132, 118]],

       [[131, 117]],

       [[131, 116]],

       [[130, 116]],

       [[129, 115]],

       [[129, 114]],

       [[126, 114]],

       [[125, 113]],

       [[125, 110]],

       [[112, 110]],

       [[111, 109]],

       [[111, 106]]], dtype=int32), array([[[ 86,  14]],

       [[ 86,  15]],

       [[ 85,  16]],

       [[ 85,  17]],

       [[ 84,  18]],

       [[ 82,  18]],

       [[ 82,  23]],

       [[ 81,  24]],

       [[ 80,  24]],

       [[ 80,  25]],

       [[ 79,  26]],

       [[ 66,  26

...

[array([[[80, 41]],

       [[80, 53]],

       [[93, 53]],

       [[93, 41]]], dtype=int32), array([[[  9,  29]],

       [[  9,  31]],

       [[  7,  33]],

       [[  6,  33]],

       [[  5,  34]],

       [[  4,  34]],

       [[  4,  91]],

       [[  3,  92]],

       [[  0,  92]],

       [[  0, 117]],

       [[  1, 117]],

       [[  2, 118]],

       [[  2, 123]],

       [[  4, 123]],

       [[  6, 125]],

       [[  6, 127]],

       [[  7, 127]],

       [[  8, 128]],

       [[  8, 136]],

       [[  9, 136]],

       [[ 10, 137]],

       [[ 10, 138]],

       [[ 11, 139]],

       [[ 25, 139]],

       [[ 25, 138]],

       [[ 26, 137]],

       [[ 29, 137]],

       [[ 29, 134]],

       [[ 30, 133]],

       [[ 43, 133]],

       [[ 44, 132]],

       [[ 45, 132]],

       [[ 46, 131]],

       [[ 47, 131]],

       [[ 47, 128]],

       [[ 48, 127]],

       [[ 49, 127]],

       [[ 49, 126]],

       [[ 50, 125]],

       [[ 63, 125]],

       [[ 64, 124]],

   

Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:150, 0:150]
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:150, 0:150]
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:150, 0:150]
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:150, 0:150]
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:150, 0:150]
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:

Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:150, 0:150]
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:150, 0:150]
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:150, 0:150]
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:150, 0:150]
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:150, 0:150]
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:

Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:150, 0:150]
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:150, 0:150]
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:150, 0:150]
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:150, 0:150]
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:150, 0:150]
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:

Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:150, 0:150]
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:150, 0:150]
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:150, 0:150]
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:150, 0:150]
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0:150, 0:150]
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "<ipython-input-9-9f812fb6c9f1>", line 19, in <module>
    roi = frame[0: